# Fester R-CNN 

selective Search를 사용해 계산했던 Region Proposal 단계를 Neural Network 안으로 포함 

진정한 end-to-end Object Detection 모델 제시

성능 : pascal VOC을 기준으로 78.8%

핵심 idea - Region Proposal network (RPN)

기존 Fest R-CNN 구조를 계승하면서 selective search를 제거하고 RPN을 통해서 RoI를 계산

이를 통해 GPU를 통한 ROI 계산이 가능 and 정확도도 높임


 Object-Detection을 수행하는 알고리즘 

1. feature Map을 먼저 추출한 다음 이를 RPN에 전달하여 ROI 계산 
ROI pooling을 진행한 다음 classificaiotn을 진행하여 Object Detection을 수행한다.


### Region ProPosal Network

1. CNN을 통해 뽑아낸 피쳐 맵을 입력으로 받는다. 이때 피쳐맵의 크기를 H x W x C로 잡는다. (Height, width, channel(color Channel))

2. 피쳐맵에 3x3 컨볼루션을 256, 512 채널만큼 수행한다. 이때 padding은 1로 설정해주어 h x W 가 보존될 수 있도록 한다.

3. 두번째 피쳐맵을 입력받아서 classidication 과 BoundingBox Regression 예측 값을 계산해 주어야함. 
이때 사용되는 Layer는 Fully Connect Layer가 아니라 1 x 1 convolution을 이용하여 계산하는 Fully convolution network의 특징을 갖는다.

4. 먼저 classidication을 수행하기 위해 1x1 컨볼루션을 2(object여부) x 9(앵커 수) channel 수만큼 수행하며, 그 결과로 H x W x 18 크기의 피쳐 맵을 얻는다. 
각 18개의 channel은 각각 해당 좌표를 앵커로 삼아 k개의 앵커 좌표들에 대한 예측값을 갖고 있다. 이 값들을 적절히 reshape 해준다음 softmax를 적절히 사용하여 오브젝트 일 확률값을 얻는다.

5.두 번째로 BoundingBoxRegression 예측 값을 얻기위한 1x1컨볼루션을 4x9 channel 수 만큼 수행한다.

6.앞서 얻은 값들로 ROI를 계산해야한다. classidication을 통해서 얻은 물체일 확률 값들을 정렬한다음 높은 순으로 k개의 앵커만 추려낸다.
그 다음 k개의 앵커들에게 각각 BoundingBox Regression을 적용해준다. 그 후, Non-Maximun suppression을 적용하여 ROI를 구한다.


### Loss Function 

두가지 task에서 얻은 Loss 값을 엮은 형태를 취한다.

    L({p_i},{t_i}) = 1/(N_cls)sigma_i(L_cls(p_i, p^*_i)) + lambda/(N_reg)sigma_i(p^*_ *L_reg(t_i, t^*_i))

p_i 해당 앵커가 object일 확률, t_i 박스 조정 값 백터

p^*_i, t^*_i ground truth label


classification은 log loss를 통해 계산하고 
regression loss는 smoothL1 함수를 사용해 계산한다.

또한 N_cls, N_reg 로 나누어 주는 부분이 있는데 이는 특별한 의미를 갖는것은 아니고

N_cls는 minibatch 사이즈이며 Nreg는 앵커 개수에 해당한다. 
람다는 classification loss와 regression loss 사이의 가중 치를 조정해주는 부분 (논문에는 10으로 설정되어있어 사실상 동일 가중치 )


### Training 

전체 모델을 한번에 학습시키기 매우 어렵다. RPN이 제대로 ROI를 계산해 내지 못하기 때문에 뒷단의 
Classification 레이어가 제대로 학습되지 못한다. 

여기서는 4단계에 걸처서 모델을 번갈아서 학습시키는 Alternating Training기법을 취한다.

1. ImageNet pretrained 모델을 불러온다음 RPN을 학습시킨다.
2. 1단계에서 학습시킨 RPN에서 기본 CNN을 제외한 Region Proposal 레이어만 갖고온다. 이를 활용하여 fast RCNN 을 학습시킨다.
이때, 처음 피쳐맵을 추출하는 CNN까지 fine tune 시킨다.
3. 앞서 학습시킨 Fast RCNN과 RPN을 불러온 다음, 다른 웨이트들은 고정하고 RPN에 해당하는 레이어들만 fine tune 시킨다.
여기서 부터 RPN과 Fast RCNN이 컨볼루션 웨이트를 공유하게 된다.
4. 마지막으로 공유하는 CNN과 RPN은 고정시킨채 Fast R-CNN에 해당하는 레이어만 Fine tune 시킨다.

